In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from subprocess import check_output

import warnings
warnings.filterwarnings("ignore")

data_frame =pd.read_csv('../input/stock-price-prediction-with-rnn/stock_data.csv')

In [ ]:
data_frame.head()

In [ ]:
data_frame.tail()

In [ ]:
data_frame.shape

In [ ]:
list(data_frame)

In [ ]:
data_frame.columns

In [ ]:
data_frame.isnull().any()

In [ ]:
data_frame.isnull().sum()

In [ ]:
data_frame.describe()

In [ ]:
data_frame.info()

In [ ]:
data_frame.describe(include = "all").T

In [ ]:
data_frame['Date'] = pd.to_datetime(data_frame['Date'])

In [ ]:
print(f'Dataframe comtains stock prices between {data_frame.Date.min()} {data_frame.Date.max()}')
print(f'Total days = {(data_frame.Date.max() - data_frame.Date.min()).days}days')

In [ ]:
len(data_frame)-len(data_frame.drop_duplicates())

In [ ]:
data_frame["Change"] = data_frame["Close"] - data_frame["Open"]
df_interest = data_frame[["Stock", "Date", "Open", "Close", "Change", "Volume"]]
df_interest["Date"] = pd.to_datetime(df_interest["Date"])
df_interest.head()

In [ ]:
symbolCloseWide = data_frame.pivot(index='Stock', 
                                  columns='Date', 
                                  values='Close')
symbolCloseWide.head()

In [ ]:
symbols = df_interest["Stock"].unique().tolist()
for u in symbols[:10]:
    dates = df_interest[(df_interest["Stock"] == u)]["Date"]
    values = df_interest[(df_interest["Stock"] == u)]["Close"]
    plt.plot(dates.tolist(), values.tolist())
plt.legend(symbols, loc='upper left')
plt.show()

In [ ]:
data_frame.corr()

In [ ]:
import seaborn as sns
plt.figure(figsize=(12,6))
autoDataColumns = ['Open','High','Low','Close','Volume']
sns.heatmap(data_frame[autoDataColumns].corr(), annot=True, fmt='.6f', linewidths=.5)

In [ ]:
#Mask by most recent year
latest_year = max(pd.unique(list(df_interest["Date"].apply(lambda x:x.year))))
latest_year_mask = [i==latest_year for i in df_interest["Date"].apply(lambda x:x.year)]
df_masked1 = df_interest[latest_year_mask]
df_masked1.head()

In [ ]:
#Mask by most top 10 closers
symbols = df_interest["Stock"].unique().tolist()
maxPerSymbol={}
for u in symbols:
    values = df_interest[(df_interest["Stock"] == u)]["Close"].tolist()
    maxPerSymbol[u] = max(values)

top10Closers = list(dict(sorted(maxPerSymbol.items(), 
                   key=lambda v:v[1], #sort by key -> v[0] | sort by value->v[1]
                   reverse=True)[:10]).keys())

top_10_mask = [i in top10Closers for i in df_masked1["Stock"].tolist()]
df_masked2 = df_masked1[top_10_mask]
df_masked2.head()

In [ ]:
symbols = df_masked2["Stock"].unique().tolist()
for u in symbols[:10]:
    dates = df_masked2[(df_interest["Stock"] == u)]["Date"]
    values = df_masked2[(df_interest["Stock"] == u)]["Close"]
    plt.plot(dates.tolist(), values.tolist())
#plt.legend(symbols, loc='upper left')
plt.legend(symbols, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title(r'Top 10 Closing Stocks', fontsize=16)

plt.xlabel('Time (years)', fontsize=12)
plt.ylabel('Price ($/stock)', fontsize=12)
plt.show()